In [20]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
X = np.array([
    [22, 60, 1],
    [25, 75, 2],
    [30, 80, 3],
    [35, 120, 5],
    [42, 150, 10],
    [50, 110, 3],
    [23, 95, 1],
    [29, 100, 2],
    [33, 105, 4],
    [48, 135, 9]
]
)


y = np.array([0, 0, 1, 1, 1, 1, 0, 1, 1, 1])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
model = Pipeline([
    ("scaler", MinMaxScaler()),
    ("knn", KNeighborsClassifier(3))
])
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

1.0

In [14]:
scaler = model.named_steps["scaler"]
new_c = np.array([27, 95, 3]).reshape(1, -1)
new_scaled = scaler.transform(new_c)
new_scaled

array([[0.19230769, 0.38888889, 0.22222222]])

In [15]:
new_pred = model.predict(new_scaled)
new_pred[0]

np.int64(0)

In [21]:
new_client = np.array([[27, 95, 3]])
X_scaled = model.named_steps["scaler"].transform(X)
new_scaled = model.named_steps["scaler"].transform(new_client)

distances = euclidean_distances(X_scaled, new_scaled).flatten()
nearest_indices = distances.argsort()[:3]

first = X[nearest_indices[0]], y[nearest_indices[0]], distances[nearest_indices[0]]
second = X[nearest_indices[1]], y[nearest_indices[1]], distances[nearest_indices[1]]
third = X[nearest_indices[2]], y[nearest_indices[2]], distances[nearest_indices[2]]

print("1st:", first)
print("2nd:", second)
print("3rd:", third)

1st: (array([ 29, 100,   2]), np.int64(1), np.float64(0.14611385467759627))
2nd: (array([30, 80,  3]), np.int64(1), np.float64(0.2027101064210499))
3rd: (array([25, 75,  2]), np.int64(0), np.float64(0.26008759067868265))


In [22]:
votes = y[nearest_indices]
prediction = 1 if sum(votes) >= 2 else 0
print("Prediction:", prediction)

Prediction: 1


המודל חישב את המרחקים בין הלקוח החדש לכל שאר הלקוחות לאחר נרמול הנתונים

שלושת השכנים הקרובים ביותר הם אחד שהחזיר הלוואה ושניים שלא לפי שיטת KNN עם K=3

תחזית המודל מבוססת על רוב הקולות
ולכן התחזית היא לא יחזיר הלוואה